In [ ]:
!pip install transformers datasets scikit-learn pandas torch
!pip install git+https://github.com/huggingface/peft.git


In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments, EvalPrediction
import torch
import time
from peft import LoraConfig, TaskType, get_peft_model

/Users/dervlagargan/Documents/College/CS6514/AI/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"

df = pd.read_csv("data/dataset.csv")
df["humor"] = df["humor"].astype(int)
print(df.head())

train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["humor"])
train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)

                                                text  humor
0  Joe biden rules out 2020 bid: 'guys, i'm not r...      0
1  Watch: darvish gave hitter whiplash with slow ...      0
2  What do you call a turtle without its shell? d...      1
3      5 reasons the 2016 election feels so personal      0
4  Pasco police shot mexican migrant from behind,...      0


In [5]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

def tokenize_fn(batch):
    return tokenizer(batch["text"], truncation=True, padding="max_length", max_length=64)

train_ds = train_ds.map(tokenize_fn, batched=True)
test_ds = test_ds.map(tokenize_fn, batched=True)
train_ds = train_ds.rename_column("humor", "labels")
test_ds = test_ds.rename_column("humor", "labels")

train_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])
test_ds.set_format(type="torch", columns=["input_ids", "attention_mask", "labels"])


Map: 100%|██████████| 40000/40000 [00:06<00:00, 6189.90 examples/s]


In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2).to(device)

r = 4
lora_config = LoraConfig(task_type=TaskType.SEQ_CLS, 
                         r=r,
                         lora_alpha=1, 
                         lora_dropout=0.1)

model = get_peft_model(model, lora_config)


def compute_metrics(p: EvalPrediction):
    preds = p.predictions.argmax(-1)
    labels = p.label_ids
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds),
        "precision": precision_score(labels, preds),
        "recall": recall_score(labels, preds)
    }

training_args = TrainingArguments(
    output_dir="./lora-bert-humor",
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=3,
    logging_dir="./logs",
    logging_steps=50,
    fp16=False,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    compute_metrics=compute_metrics,
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: LoraConfig.__init__() got an unexpected keyword argument 'lora_lr_ratio'

In [ ]:
start_time = time.time()
if torch.cuda.is_available():
    torch.cuda.reset_peak_memory_stats()
trainer.train()
end_time = time.time()

if torch.cuda.is_available():
    peak_gpu_memory_gb = torch.cuda.max_memory_allocated() / 1024**3 

results = trainer.evaluate()

print(f"Training time: {end_time - start_time:.2f} seconds")
if torch.cuda.is_available():
    print(f"Peak GPU memory usage: {peak_gpu_memory_gb:.2f} GB")
print("Performance metrics:", results)

/Users/dervlagargan/Documents/College/CS6514/AI/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.128100,0.084559,0.974550,0.974663,0.970364,0.979000
2,0.070600,0.071810,0.979625,0.979630,0.979409,0.979850
3,0.087600,0.069996,0.980325,0.980312,0.980974,0.979650


/Users/dervlagargan/Documents/College/CS6514/AI/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/dervlagargan/Documents/College/CS6514/AI/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/Users/dervlagargan/Documents/College/CS6514/AI/.venv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:684: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Training time: 7249.16 seconds
Performance metrics: {'eval_loss': 0.06999601423740387, 'eval_accuracy': 0.980325, 'eval_f1': 0.980311710404523, 'eval_precision': 0.9809743153256897, 'eval_recall': 0.97965, 'eval_runtime': 232.5448, 'eval_samples_per_second': 172.01, 'eval_steps_per_second': 21.501, 'epoch': 3.0}
